In [1]:
from sklearn.datasets import load_diabetes
import pickle
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import os
import torch


from synthcity.metrics.eval_performance import (
    PerformanceEvaluatorMLP,
    PerformanceEvaluatorXGB,
)
from synthcity.utils import reproducibility
from synthcity.plugins import Plugins
import synthcity.logger as log
from synthcity.plugins.core.dataloader import GenericDataLoader
from DGE_utils import metric_different_datasets, mean_across_pandas, add_std, get_folder_names

reproducibility.clear_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


assert device.type == 'cuda'
Plugins(categories=["generic"]).list()

[2023-12-17T13:46:04.114081+0000][31803][CRITICAL] load failed: 
GReaT is not installed. Please install it with pip install GReaT.
Please be aware that GReaT is only available for python >= 3.9.

[2023-12-17T13:46:04.115400+0000][31803][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-17T13:46:04.116063+0000][31803][CRITICAL] module plugin_great load failed
[2023-12-17T13:46:04.123670+0000][31803][CRITICAL] module disabled: /home/JacquelineYau/deep_generative_ensemble/synthcity/src/synthcity/plugins/generic/plugin_goggle.py


['marginal_distributions',
 'ctgan',
 'dummy_sampler',
 'uniform_sampler',
 'ddpm',
 'rtvae',
 'bayesian_network',
 'tvae',
 'nflow',
 'arf']

In [2]:
# Set up params for boosting experiment
from DGE_data import get_real_and_synthetic, get_real_and_synthetic_with_multiple_models

# let's restrict ourselves to classification datasets
datasets = ['moons', 'circles', 'breast_cancer',
            'adult', 'covid']
# ['moons', 'circles','cal_housing', 'adult', 'diabetes', 'breast_cancer',  'seer', 'cutract' ]
model_name = 'ctgan'  # synthetic data model
# model_names = ['tvae', 'nflow', 'ctgan', 'adsgan']
# model_name = model_names[0]
# for m in model_names[1:]:
#     model_name += "_" + m

n_models = 20  # number of models in ensemble, for each run. 20
max_n = 2000  # maximum number of data points to use for training generative model.
nsyn = 2000  # number of synthetic data points per synthetic dataset. Defaults to same as generative training size if None

num_runs = 3 # Number of runs. Don't choose to large, since total number of synthetic datasets is num_runs*n_models. 10

# Per section 4.1, 10 runs with different seeds

# Whether to load and save models and synthetic datasets
load = True  # results
load_syn = True  # data
save = True  # save results and data

verbose = True

In [3]:
# Run boosting experiment
from DGE_experiments import predictive_experiment, predictive_experiment_stacking, boosting_DGE
import pandas as pd
import time

all_means = {}
all_stds = {}

#datasets = ['moons', 'circles', 'breast_cancer', 'adult', 'covid']
datasets = ['covid', 'breast_cancer']
test_only_dataset = ['adult']
#model_names = ['ctgan']
#num_runs=1
#model_type = ['deepish_mlp', 'largest_mlp', 'rf', 'svm', 'xgboost', 'knn']
model_type = 'deepish_mlp'
#mixed_models = True
#meta_model='lr'
boosting = "SAMME"

print("Model to run: ", model_name)
print("Downstream classifier model type: ", model_type)
print("boosting method: ", boosting)
print("n_models: ", n_models)
print("num_runs: ", num_runs)
print("datasets: ", datasets)
print("model string: ", model_name)
print("verbose: ", verbose)

start_time = time.time()
for dataset in datasets:
    workspace_folder, results_folder = get_folder_names(
        dataset, model_name, max_n=max_n, nsyn=nsyn)
    # For toy runs
    workspace_folder = f"{workspace_folder}_{boosting}_observe"
    results_folder = f"{results_folder}_{boosting}_observe"

    print(f'Dataset {dataset}\n')
    print("workspace_folder: ", workspace_folder)
    print("results_folder: ", results_folder)

    means, stds, _ = boosting_DGE(dataset, model_name, num_runs=num_runs, num_iter=n_models, boosting=boosting, p_train=0.8, 
                                  max_n=max_n, nsyn=nsyn, reduce_to=20000, task_type=model_type, workspace_folder=workspace_folder, 
                                  save=save, load=load, verbose=verbose)
    print("printing weighted avg means to latex:")
    print(means.to_latex())
    print("printing weighted stds to latex:")
    print(stds.to_latex())

    all_means[dataset] = means
    all_stds[dataset] = stds

    print("size of means: ", means.shape)
    print("mean elements: ", means)

end_time = time.time()
time_elapsed = end_time - start_time
print("Time it took to run the experiment: ", time_elapsed)

Model to run:  ctgan
Downstream classifier model type:  deepish_mlp
boosting method:  SAMME
n_models:  20
num_runs:  3
datasets:  ['covid', 'breast_cancer']
model string:  ctgan
verbose:  True
Dataset covid

workspace_folder:  workspace/covid/ctgan/nmax_2000_nsyn_2000_SAMME_observe
results_folder:  results/covid_ctgan_nmax_2000_nsyn_2000_SAMME_observe
Boosting DGE


[2023-12-17T13:57:22.594172+0000][31803][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-17T13:57:22.595050+0000][31803][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-17T13:57:22.595844+0000][31803][CRITICAL] module plugin_great load failed
[2023-12-17T13:57:22.596629+0000][31803][CRITICAL] module disabled: /home/JacquelineYau/deep_generative_ensemble/synthcity/src/synthcity/plugins/generic/plugin_goggle.py


np.unique(y):  [1 2]
n_train: 2000, nsyn: 2000
targettype:  classification
Run 1 / 3
Boosting iter: 1 / 20
Boost generative model training and synthetic data generation
Training model 1/20
random state: 0


[2023-12-17T13:57:23.613962+0000][31803][CRITICAL] load failed: libcudart.so.12: cannot open shared object file: No such file or directory
[2023-12-17T13:57:23.615347+0000][31803][CRITICAL] load failed: module 'synthcity.plugins.privacy.plugin_decaf' has no attribute 'plugin'
[2023-12-17T13:57:23.616018+0000][31803][CRITICAL] module plugin_decaf load failed


self._plugins:  {'uniform_sampler': <class 'synthcity.plugins.generic.plugin_uniform_sampler.UniformSamplerPlugin'>, 'bayesian_network': <class 'synthcity.plugins.generic.plugin_bayesian_network.BayesianNetworkPlugin'>, 'ctgan': <class 'synthcity.plugins.generic.plugin_ctgan.CTGANPlugin'>, 'marginal_distributions': <class 'synthcity.plugins.generic.plugin_marginal_distributions.MarginalDistributionPlugin'>, 'dummy_sampler': <class 'synthcity.plugins.generic.plugin_dummy_sampler.DummySamplerPlugin'>, 'arf': <class 'synthcity.plugins.generic.plugin_arf.ARFPlugin'>, 'ddpm': <class 'synthcity.plugins.generic.plugin_ddpm.TabDDPMPlugin'>, 'rtvae': <class 'synthcity.plugins.generic.plugin_rtvae.RTVAEPlugin'>, 'nflow': <class 'synthcity.plugins.generic.plugin_nflow.NormalizingFlowsPlugin'>, 'tvae': <class 'synthcity.plugins.generic.plugin_tvae.TVAEPlugin'>}
self._available_plugins:  {'uniform_sampler': '/home/JacquelineYau/deep_generative_ensemble/synthcity/src/synthcity/plugins/generic/plugin

...

fileroot in aggregate:  workspace/breast_cancer/ctgan/nmax_2000_nsyn_2000_SAMME_observe/supervised_task_deepish_mlp
task function name:  supervised_task
task type:  deepish_mlp
Saving model as workspace/breast_cancer/ctgan/nmax_2000_nsyn_2000_SAMME_observe/supervised_task_deepish_mlp_DGE_17__0.pkl
loading existing model...
supervised task:  classification
model type:  deepish_mlp
shape of pred in supervised task:  (455,)
Boosting iter 17 accuracy:  0.9362637362637363
boost with SAMME
normalized data_weights:  [0.00221194 0.00219011 0.00219528 0.00220537 0.00219011 0.00219576
 0.00219011 0.00219484 0.00219011 0.00219011 0.00219011 0.00224487
 0.00222706 0.00219011 0.00225077 0.00219011 0.0021959  0.00219011
 0.00222792 0.002195   0.00219011 0.00219011 0.00219011 0.00219011
 0.00219011 0.00219011 0.00219576 0.00219011 0.00219011 0.00221248
 0.00219011 0.00221681 0.00219011 0.00219011 0.00219011 0.002195
 0.00219011 0.00219011 0.00223001 0.00221069 0.00219582 0.00219011
 0.00219011 0.0021

 70%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                               | 1399/2000 [02:42<01:09,  8.62it/s]
[2023-12-17T23:25:13.889782+0000][31803][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-17T23:25:13.891929+0000][31803][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2023-12-17T23:25:13.893632+0000][31803][CRITICAL] module plugin_great load failed
[2023-12-17T23:25:13.895736+0000][31803][CRITICAL] module disabled: /home/JacquelineYau/deep_generative_ensemble/synthcity/src/synthcity/plugins/generic/plugin_goggle.py
[2023-12-17T23:25:13.899401+0000][31803][CRITICAL] load failed: module 'synthcity.plugins.privacy.plugin_decaf' has no attribute 'plugin'
[2023-12-17T23:25:13.901124+0000][31803][CRITICAL] load failed: module 'synthcity.plugins.privacy.plugin_decaf' has no attribute 'plug

fileroot in aggregate:  workspace/breast_cancer/ctgan/nmax_2000_nsyn_2000_SAMME_observe/supervised_task_deepish_mlp
task function name:  supervised_task
task type:  deepish_mlp
Saving model as workspace/breast_cancer/ctgan/nmax_2000_nsyn_2000_SAMME_observe/supervised_task_deepish_mlp_DGE_18__0.pkl
loading existing model...
supervised task:  classification
model type:  deepish_mlp
shape of pred in supervised task:  (455,)
Boosting iter 18 accuracy:  0.9252747252747253
boost with SAMME
normalized data_weights:  [0.00221706 0.00218969 0.00219486 0.00220495 0.00218969 0.00219535
 0.00218969 0.00219442 0.00218969 0.00218969 0.00218969 0.00225007
 0.00223222 0.00218969 0.00225598 0.00218969 0.00219549 0.00218969
 0.00223307 0.00219458 0.00218969 0.00218969 0.00218969 0.00218969
 0.00218969 0.00218969 0.00219535 0.00218969 0.00218969 0.00221206
 0.00218969 0.00221639 0.00218969 0.00218969 0.00218969 0.00219458
 0.00218969 0.00218969 0.00222959 0.00221027 0.0021954  0.00218969
 0.00218969 0.00

 45%|███████████████████████████████████████████████████████████████████████                                                                                       | 899/2000 [01:57<02:24,  7.63it/s]


fileroot in aggregate:  workspace/breast_cancer/ctgan/nmax_2000_nsyn_2000_SAMME_observe/supervised_task_deepish_mlp
task function name:  supervised_task
task type:  deepish_mlp
Saving model as workspace/breast_cancer/ctgan/nmax_2000_nsyn_2000_SAMME_observe/supervised_task_deepish_mlp_DGE_19__0.pkl
loading existing model...
supervised task:  classification
model type:  deepish_mlp
shape of pred in supervised task:  (455,)
Boosting iter 19 accuracy:  0.9340659340659341
boost with SAMME
normalized data_weights:  [0.00222252 0.0021893  0.00219448 0.00220456 0.0021893  0.00219496
 0.0021893  0.00219403 0.0021893  0.0021893  0.0021893  0.0022556
 0.00223182 0.0021893  0.00226153 0.0021893  0.0021951  0.0021893
 0.00223268 0.00219419 0.0021893  0.0021893  0.0021893  0.0021893
 0.0021893  0.0021893  0.00220075 0.0021893  0.0021893  0.00221167
 0.0021893  0.002216   0.0021893  0.0021893  0.0021893  0.00219419
 0.0021893  0.0021893  0.00223507 0.00220988 0.00219501 0.0021893
 0.0021893  0.002189

In [4]:
# Metrics
# Print results, aggregated over different datasets
means_consolidated = metric_different_datasets(all_means, to_print=False)
if num_runs>1:
    stds_consolidated = metric_different_datasets(all_stds, to_print=False)
    stds_consolidated.drop(columns=['Mean'], inplace=True)
    print(add_std(means_consolidated, stds_consolidated).to_latex())
else:
    print(means_consolidated.to_latex())

\begin{tabular}{llll}
\toprule
{} &  Breast Cancer &       COVID-19 &   Mean \\
\midrule
Oracle            &    0.993 ± 0.0 &  0.928 ± 0.001 &  0.961 \\
Naive (S)         &  0.973 ± 0.003 &     0.82 ± 0.0 &  0.897 \\
Naive (E)         &  0.982 ± 0.002 &  0.861 ± 0.002 &  0.922 \\
DGE\$\_\{5\}\$         &    0.983 ± 0.0 &    0.915 ± 0.0 &  0.949 \\
DGE\$\_\{10\}\$        &    0.986 ± 0.0 &    0.933 ± 0.0 &   0.96 \\
DGE\$\_\{20\}\$        &    0.984 ± 0.0 &    0.939 ± 0.0 &  0.961 \\
DGE\$\_20\$ (concat) &    0.97 ± 0.01 &  0.872 ± 0.013 &  0.921 \\
\bottomrule
\end{tabular}

